# Prepare the features and data pipeline for general machine learning study:
    Load the merged data into python pandas
    Build up the data structurte for machine learning algorithms, clutering in this experimental
    Apply DB Scan, Set Level Tree, etc

In [2]:
import pandas as pd
import os
import csv

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn

sns.set(style="darkgrid")

import sys
print (sys.path)

['', '/opt/mapr/spark/spark-2.1.0/python', '/opt/mapr/spark/spark-2.1.0/python/lib/py4j-0.9-src.zip', '/mapr/datalake/optum/optuminsight/health_at_scale/python/lib/python3.5/site-packages', '/opt/mapr/tools/python/anaconda3/lib/python35.zip', '/opt/mapr/tools/python/anaconda3/lib/python3.5', '/opt/mapr/tools/python/anaconda3/lib/python3.5/plat-linux', '/opt/mapr/tools/python/anaconda3/lib/python3.5/lib-dynload', '/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages', '/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/Sphinx-1.4.6-py3.5.egg', '/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/setuptools-27.2.0-py3.5.egg', '/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/IPython/extensions', '/home/hatscale/.ipython']


In [3]:
data_dir = '/mapr/datalake/uhclake/perf/Sqoop_RX/ymalcow/101_clustering_proj01/'
path_csvs = 'prep01ML_all2_csv'

path_mrgs = os.path.join(data_dir, path_csvs)

In [4]:
# mem_attrs: member attributes
# mem_attrs: merged claim attributes with members
mem_attrs = ['member_individual_identifier','src_cdb_xref_id','lgcy_src_id','social_security_number_hashed' , \
             'member_last_name','member_first_name' ,'member_middle_name','member_middle_initial_text', \
             'member_name_suffix', 'mem_dob_hashed', 'memeber_zip_code_of_residence','member_sex','addr_line_1',\
             'addr_line_2','city_name','state','zipcode','country']
mrg_attrs = ['claim_identifier', 'claim_system_identifier', 'member_identifier', 'member_individual_identifier', \
             'member_division_identifier', 'admission_src_code', 'icd_version_number', 'procedure_code', \
             'diag_code_diag_type_present_on_admis_ind', 'provider_medicare_identifier', 'drg', 'first_service_date', \
             'last_service_date', 'charged_amount', 'paid_amount', 'network_type', 'revenue_code', 'admission_type_code', \
             'hospital_discharge_code', 'place_of_service_code', 'billing_npi_identifier', 'rendering_npi_identifier', \
             'attending_npi_identifier', 'referring_npi_identifier', 'provider_mpin_number', 'provider_tin_identifier', \
             'provider_name', 'provider_address_street_text', 'provider_city_name', 'provider_state_code', \
             'provider_zip_code', 'mth', 'member_last_name', 'member_first_name', 'member_middle_name', \
             'member_middle_initial_text', 'member_name_suffix', 'member_age', 'member_zip_code_of_residence', \
             'member_income', 'member_income_type', 'member_alive_ind', 'tops_relationship_code', 'tops_sequence_number', \
             'member_marital_status', 'member_sex', 'member_race', 'member_coverage_effective_date', 
             'member_coverage_end_date', 'member_cosmos_panel_number', 'government_program_type_description', \
             'employee_status_type_description']

In [5]:
len(mrg_attrs)

52

In [15]:
# Define the list of attributes slected for machine learning study
ML_F_set1 = ['network_type', 'place_of_service_code', 'mth', 'member_age', \
             'member_sex', 'government_program_type_description', 'employee_status_type_description', 
             'charged_amount', 'paid_amount', 'member_income']

mrgs = os.listdir(path_mrgs)
mrgs

['.000029_0.crc',
 '000008_0',
 '000018_0',
 '000002_0',
 '.000013_0.crc',
 '.000005_0.crc',
 '000011_0',
 '000001_0',
 '.000019_0.crc',
 '.000043_0.crc',
 '.000042_0.crc',
 '.000048_0.crc',
 '.000000_0.crc',
 '.000026_0.crc',
 '.000028_0.crc',
 '000048_0',
 '.000014_0.crc',
 '.000015_0.crc',
 '000020_0',
 '000022_0',
 '000038_0',
 '000030_0',
 '.000017_0.crc',
 '000017_0',
 '000028_0',
 '000032_0',
 '.000007_0.crc',
 '.000022_0.crc',
 '000036_0',
 '.000044_0.crc',
 '.000020_0.crc',
 '.000006_0.crc',
 '000044_0',
 '000027_0',
 '000004_0',
 '000013_0',
 '000006_0',
 '000039_0',
 '000015_0',
 '000005_0',
 '000016_0',
 '000019_0',
 '.000024_0.crc',
 '.000025_0.crc',
 '.000009_0.crc',
 '000034_0',
 '000047_0',
 '000000_0',
 '000026_0',
 '000035_0',
 '.000012_0.crc',
 '000043_0',
 '.000038_0.crc',
 '000024_0',
 '.000002_0.crc',
 '.000030_0.crc',
 '000012_0',
 '.000036_0.crc',
 '.000047_0.crc',
 '.000034_0.crc',
 '.000039_0.crc',
 '.000046_0.crc',
 '.000041_0.crc',
 '000023_0',
 '.000001_0.c

In [16]:
# Read in the data chunk by chunk, select the slected features and concat the dataframe
df_all = pd.DataFrame(columns = ML_F_set1)
cnt = 0
for chunk in mrgs:
    if chunk[-3:] == 'crc':
        continue
    cnt += 1
    curr = pd.read_csv(os.path.join(path_mrgs, chunk), sep = ';', names = mrg_attrs, error_bad_lines=False)
    curr_slt = curr[ML_F_set1]
    print ("Chunk {} contains {} records".format(cnt, len(curr_slt.index)))
    df_all = pd.concat([df_all, curr_slt])    

/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13,16,19,24,30,37,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Chunk 1 contains 396521 records


/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13,16,19,24,30,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Chunk 2 contains 393846 records
Chunk 3 contains 786582 records
Chunk 4 contains 395082 records
Chunk 5 contains 787528 records
Chunk 6 contains 387402 records
Chunk 7 contains 393326 records
Chunk 8 contains 392903 records
Chunk 9 contains 390089 records
Chunk 10 contains 392028 records
Chunk 11 contains 393864 records
Chunk 12 contains 392194 records
Chunk 13 contains 391676 records
Chunk 14 contains 390286 records
Chunk 15 contains 389436 records
Chunk 16 contains 392417 records
Chunk 17 contains 780634 records
Chunk 18 contains 394657 records
Chunk 19 contains 396608 records
Chunk 20 contains 390007 records
Chunk 21 contains 394160 records
Chunk 22 contains 396788 records
Chunk 23 contains 394037 records


/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13,16,19,24,30,36,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Chunk 24 contains 393292 records
Chunk 25 contains 391116 records
Chunk 26 contains 387695 records
Chunk 27 contains 788625 records
Chunk 28 contains 392300 records
Chunk 29 contains 390605 records
Chunk 30 contains 389587 records
Chunk 31 contains 392536 records
Chunk 32 contains 394646 records
Chunk 33 contains 392777 records
Chunk 34 contains 389004 records
Chunk 35 contains 392009 records
Chunk 36 contains 393080 records
Chunk 37 contains 395384 records
Chunk 38 contains 391171 records
Chunk 39 contains 389688 records
Chunk 40 contains 396471 records
Chunk 41 contains 390392 records
Chunk 42 contains 392031 records
Chunk 43 contains 394455 records
Chunk 44 contains 392425 records
Chunk 45 contains 781596 records
Chunk 46 contains 395283 records
Chunk 47 contains 389861 records
Chunk 48 contains 389889 records
Chunk 49 contains 388250 records


In [17]:
print (len(df_all.index), '\n', df_all.dtypes)
df_all.head()

21186239 
 network_type                            object
place_of_service_code                   object
mth                                     object
member_age                              object
member_sex                              object
government_program_type_description     object
employee_status_type_description        object
charged_amount                          object
paid_amount                            float64
member_income                          float64
dtype: object


,network_type,place_of_service_code,mth,member_age,member_sex,government_program_type_description,employee_status_type_description,charged_amount,paid_amount,member_income
0,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.01,0.0,0.0
1,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.05,0.0,0.0
2,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.02,0.0,0.0
3,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,130,0.0,0.0
4,IN,11,2018-01,57,MALE,MEDICARE,ACTIVE,205,0.0,0.0


In [18]:
# save the merged dataframe with selected effective attributes to csv file
out_ml = 'ml2018_set1.csv'
path_out_ml = os.path.join(data_dir, out_ml)

df_all.to_csv(path_out_ml, sep = ';')

In [19]:
os.listdir(data_dir)

['tables_proc', 'ml2018_set1.csv', 'prep01ML_all2_csv']

In [22]:
df_all.charged_amount.unique()

array([0.01, 0.05, 0.02, ..., 1518.96, 5200.6, 57354.3], dtype=object)

In [23]:
df_all[df_all.charged_amount == '2018-01-11']

,network_type,place_of_service_code,mth,member_age,member_sex,government_program_type_description,employee_status_type_description,charged_amount,paid_amount,member_income
115952,63.13,,019151705,NaN,\N,8,MEDICARE,2018-01-11,70.0,1876.0
224659,63.13,,019151705,NaN,\N,49,MEDICARE,2018-01-11,70.0,2916.0
168369,63.13,,019151705,NaN,\N,112,MEDICARE,2018-01-11,70.0,34957.0
214277,63.13,,019151705,NaN,\N,49,MEDICARE,2018-01-11,70.0,1876.0
47441,40.03,,019151705,NaN,\N,112,MEDICARE,2018-01-11,99.0,1569.0
219476,132.69,,019151705,NaN,\N,49,MEDICARE,2018-01-11,199.0,1519.0
165188,191.06,,019151705,NaN,\N,112,MEDICARE,2018-01-11,211.0,2370.0
142327,73.87,,019151705,NaN,\N,112,MEDICARE,2018-01-11,129.0,2360.0
169535,0.00,,019151705,NaN,\N,112,MEDICARE,2018-01-11,99.0,23919.0
226621,137.00,,019151705,NaN,\N,8,MEDICARE,2018-01-11,199.0,1507.0


In [24]:
df_all.member_sex.unique()

array(['FEMALE', 'MALE', '\\N'], dtype=object)

In [34]:
df_all = df_all.reindex(columns = ML_F_set1)
df_all.columns

Index(['network_type', 'place_of_service_code', 'mth', 'member_age',
       'member_sex', 'government_program_type_description',
       'employee_status_type_description', 'charged_amount', 'paid_amount',
       'member_income'],
      dtype='object')

In [36]:
df_all.head()

,network_type,place_of_service_code,mth,member_age,member_sex,government_program_type_description,employee_status_type_description,charged_amount,paid_amount,member_income
0,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.01,0.0,0.0
1,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.05,0.0,0.0
2,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.02,0.0,0.0
3,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,130,0.0,0.0
4,IN,11,2018-01,57,MALE,MEDICARE,ACTIVE,205,0.0,0.0


In [37]:
df_all['charged_amount'] = df_all['charged_amount'].astype(str)

In [38]:
df_all.dtypes

network_type                            object
place_of_service_code                   object
mth                                     object
member_age                              object
member_sex                              object
government_program_type_description     object
employee_status_type_description        object
charged_amount                          object
paid_amount                            float64
member_income                          float64
dtype: object

In [41]:
df_all['charged_amount'] = df_all.charged_amount.apply(lambda x : str(x) if pd.notnull(x) else '')

In [43]:
print (df_all[df_all.charged_amount.str.contains('2018')]['charged_amount'].count())
df_all[df_all.charged_amount.str.contains('2018')]

543


,network_type,place_of_service_code,mth,member_age,member_sex,government_program_type_description,employee_status_type_description,charged_amount,paid_amount,member_income
20893,ADJIN,11,2018-01,76,FEMALE,NON-GOVERNMENTAL,ACTIVE,2018.0,29.90,0.0
116723,63.13,,019151705,NaN,\N,8,MEDICARE,2018-01-13,70.00,1876.0
137280,74.46,,019151705,NaN,\N,112,MEDICARE,2018-01-08,129.00,1701.0
147053,IN,22,2018-01,71,MALE,MEDICARE,ACTIVE,2018.00,844.82,0.0
147054,IN,22,2018-01,71,MALE,MEDICARE,ACTIVE,2018.00,0.00,0.0
171735,20.72,,019151705,III,\N,112,MEDICARE,2018-01-22,199.00,2703.0
176015,IN,11,2018-01,75,FEMALE,MEDICARE,ACTIVE,2018.00,810.88,0.0
220712,91.96,,019151705,NaN,\N,49,MEDICARE,2018-01-21,199.00,1860.0
221323,69.56,,019151705,NaN,\N,49,MEDICARE,2018-03-04,129.00,5753.0
231075,76.27,,019151705,NaN,\N,8,MEDICARE,2018-03-15,164.00,1949.0


In [45]:
df_all = df_all[~df_all.charged_amount.str.contains('2018')]

In [46]:
len(df_all.index)

21185696

In [47]:
df_all.charged_amount.astype(float)

0            0.01
1            0.05
2            0.02
3          130.00
4          205.00
5           20.00
6           60.00
7         5121.00
8          360.00
9          360.00
10         190.00
11         257.00
12         129.00
13         219.00
14         350.00
15         170.00
16         165.00
17          90.00
18        1635.90
19         235.00
20         293.04
21         660.00
22         235.20
23         192.00
24         843.29
25         310.00
26         859.00
27         400.00
28         260.00
29         337.00
           ...   
388220     170.00
388221     175.00
388222      97.00
388223     235.00
388224     351.00
388225      19.00
388226     275.00
388227     118.57
388228     184.00
388229      52.00
388230     190.00
388231     180.00
388232     153.95
388233     117.00
388234     138.00
388235      66.00
388236      95.00
388237     328.33
388238     551.00
388239     484.00
388240     204.00
388241     235.00
388242     333.00
388243     181.00
388244    

# Data exploration
1. Numeric fetures - cross correlation table using seaborn

In [48]:
NUM_F = ['member_income', 'charged_amount', 'paid_amount']

In [49]:
g = sns.pairplot(df_all[NUM_F])

In [ ]:
plt.show()